In [1]:
import pandas as pd
import numpy as np
import joblib
import warnings
warnings.filterwarnings('ignore')

In [2]:
print("="*80)
print("🔮 SCRIPT DE INFERENCIA - PREDICCIÓN PARA 2025-2S")
print("="*80)

🔮 SCRIPT DE INFERENCIA - PREDICCIÓN PARA 2025-2S


In [3]:
# Cargar modelos y encoders
print("\n📂 Cargando modelos y configuración...")
label_encoders = joblib.load('models/label_encoders.pkl')
feature_info = joblib.load('models/feature_info.pkl')


📂 Cargando modelos y configuración...


In [4]:
# Modelos disponibles
modelos_disponibles = {
    '1': ('Random Forest', 'models/random_forest_model.pkl'),
    # '2': ('SVM', 'models/svm_model.pkl'),
    '3': ('Regresión Logística', 'models/logistic_regression_model.pkl'),
    '4': ('XGBoost', 'models/xgboost_model.pkl')
}

print("\n🤖 Modelos disponibles:")
for key, (nombre, _) in modelos_disponibles.items():
    print(f"  {key}. {nombre}")


🤖 Modelos disponibles:
  1. Random Forest
  3. Regresión Logística
  4. XGBoost


In [5]:
# Seleccionar modelo
modelo_seleccionado = input("\n👉 Selecciona el modelo a usar (1-4): ").strip()

if modelo_seleccionado not in modelos_disponibles:
    print("❌ Opción inválida. Usando Random Forest por defecto.")
    modelo_seleccionado = '1'

nombre_modelo, ruta_modelo = modelos_disponibles[modelo_seleccionado]
print(f"\n✅ Modelo seleccionado: {nombre_modelo}")


✅ Modelo seleccionado: Random Forest


In [6]:
anio_objetivo = 2025
termino_objetivo = "2S"

In [7]:
# Cargar modelo
modelo = joblib.load(ruta_modelo)
print(f"✓ Modelo cargado desde '{ruta_modelo}'")

# Cargar datos de inferencia
print("\n📊 Cargando datos de inferencia...")
try:
    df_inferencia = pd.read_csv(f'models/datos_inferencia_{anio_objetivo}_{termino_objetivo}.csv')
    print(f"✓ Datos cargados: {len(df_inferencia)} registros")
except FileNotFoundError:
    print(f"❌ No se encontró el archivo 'models/datos_inferencia_{anio_objetivo}_{termino_objetivo}.csv'")
    print("   Asegúrate de haber ejecutado el script de entrenamiento primero.")
    exit()

✓ Modelo cargado desde 'models/random_forest_model.pkl'

📊 Cargando datos de inferencia...
✓ Datos cargados: 30324 registros


In [ ]:
# Verificar que tengamos los datos correctos
if len(df_inferencia) == 0:
    print("❌ No hay datos disponibles para inferencia.")
    exit()

# Extraer features
X_columns = feature_info['X_columns']
X_inferencia = df_inferencia[X_columns]

print(f"\n🔮 Realizando predicciones...")
# Predicciones
predicciones = modelo.predict(X_inferencia)
probabilidades = modelo.predict_proba(X_inferencia)[:, 1]  # Probabilidad de aprobar

# Agregar predicciones al dataframe
df_inferencia['PREDICCION'] = predicciones
df_inferencia['PROB_APROBAR'] = probabilidades
df_inferencia['PREDICCION_TEXTO'] = df_inferencia['PREDICCION'].apply(
    lambda x: 'APROBARÁ' if x == 1 else 'REPROBARÁ'
)

# Resultados
print(f"\n{'='*80}")
print(f"📊 RESULTADOS DE INFERENCIA")
print(f"{'='*80}")

print(f"\n📈 Distribución de predicciones:")
print(df_inferencia['PREDICCION_TEXTO'].value_counts())
print(f"\n  • Porcentaje predicho de aprobación: {(predicciones == 1).mean()*100:.2f}%")
print(f"  • Porcentaje predicho de reprobación: {(predicciones == 0).mean()*100:.2f}%")

print(f"\n📊 Estadísticas de probabilidades:")
print(f"  • Probabilidad promedio de aprobar: {probabilidades.mean():.4f}")
print(f"  • Probabilidad mínima: {probabilidades.min():.4f}")
print(f"  • Probabilidad máxima: {probabilidades.max():.4f}")
print(f"  • Desviación estándar: {probabilidades.std():.4f}")

# Estudiantes en riesgo (probabilidad > 0.5)
estudiantes_riesgo = df_inferencia[df_inferencia['PROB_APROBAR'] > 0.5]
print(f"\n Estudiantes en ALTA PROBABILIDAD de aprobar (prob > 0.5): {len(estudiantes_riesgo)}")

# Estudiantes en riesgo moderado (0.3 < prob <= 0.5)
estudiantes_riesgo_moderado = df_inferencia[
    (df_inferencia['PROB_APROBAR'] > 0.3) & (df_inferencia['PROB_APROBAR'] <= 0.5)
]
print(f"⚠️ Estudiantes en PROBABILIDAD MODERADA (0.3 < prob <= 0.5): {len(estudiantes_riesgo_moderado)}")

# Guardar resultados
output_file = f'models/predicciones_2025_2S_{nombre_modelo.replace(" ", "_").lower()}.csv'
df_inferencia.to_csv(output_file, index=False)
print(f"\n💾 Predicciones guardadas en '{output_file}'")

# Mostrar algunos ejemplos
print(f"\n📋 EJEMPLOS DE PREDICCIONES (primeros 10 registros):")
print("="*80)
columnas_mostrar = ['COD_ESTUDIANTE', 'COD_MATERIA_ACAD_MO', 'PREDICCION_TEXTO', 'PROB_APROBAR']
if 'COD_ESTUDIANTE' in df_inferencia.columns:
    print(df_inferencia[columnas_mostrar].head(10).to_string(index=False))
else:
    print(df_inferencia[['PREDICCION_TEXTO', 'PROB_APROBAR']].head(10).to_string(index=False))


print(f"\n{'='*80}")
print(f"✅ INFERENCIA COMPLETADA")
print(f"{'='*80}")


🔮 Realizando predicciones...

📊 RESULTADOS DE INFERENCIA

📈 Distribución de predicciones:
PREDICCION_TEXTO
APROBARÁ     22410
REPROBARÁ     7914
Name: count, dtype: int64

  • Porcentaje predicho de aprobación: 73.90%
  • Porcentaje predicho de reprobación: 26.10%

📊 Estadísticas de probabilidades:
  • Probabilidad promedio de aprobar: 0.6920
  • Probabilidad mínima: 0.0703
  • Probabilidad máxima: 0.9999
  • Desviación estándar: 0.2602

 Estudiantes en ALTA PROBABILIDAD de aprobar (prob > 0.5): 22410
⚠️ Estudiantes en PROBABILIDAD MODERADA (0.3 < prob <= 0.5): 4396

💾 Predicciones guardadas en 'models/predicciones_2025_2S_random_forest.csv'

📋 EJEMPLOS DE PREDICCIONES (primeros 10 registros):
PREDICCION_TEXTO  PROB_APROBAR
        APROBARÁ      0.963844
        APROBARÁ      0.851829
        APROBARÁ      0.948531
        APROBARÁ      0.991909
        APROBARÁ      0.831589
        APROBARÁ      0.935227
        APROBARÁ      0.967770
        APROBARÁ      0.953369
        APROBARÁ 